INF8770 Technologies multimédias

Polytechnique Montréal

Exemple de codage Huffman

In [1]:
import numpy as np
from anytree import Node, RenderTree, PreOrderIter, AsciiStyle

Message à coder

In [2]:
Message = "EBFFBEABEFCFDEBBFFFEFEFCCBFACBBABCDACDCABABBBCDDCDCAAABBBABABAABAAB"

Préparation pour la création de l'arbre. On trouve les feuilles (symboles) et leurs poids (nb occurences).

In [3]:
#Liste qui sera modifié jusqu'à ce qu'elle contienne seulement la racine de l'arbre
ArbreSymb =[[Message[0], Message.count(Message[0]), Node(Message[0])]] 
#dictionnaire obtenu à partir de l'arbre.
dictionnaire = [[Message[0], '']]
nbsymboles = 1

#Recherche des feuilles de l'arbre
for i in range(1,len(Message)):
    if not list(filter(lambda x: x[0] == Message[i], ArbreSymb)):
        ArbreSymb += [[Message[i], Message.count(Message[i]),Node(Message[i])]]
        dictionnaire += [[Message[i], '']]
        nbsymboles += 1
        
longueurOriginale = np.ceil(np.log2(nbsymboles))*len(Message)    

Affichage des feuilles trouvées. Les feuilles sont triées pour la construction de l'arbre selon l'algorithme Huffman 

In [4]:
ArbreSymb = sorted(ArbreSymb, key=lambda x: x[1])
print(ArbreSymb)

[['E', 6, Node('/E')], ['D', 6, Node('/D')], ['F', 10, Node('/F')], ['C', 10, Node('/C')], ['A', 15, Node('/A')], ['B', 20, Node('/B')]]


Création de l'arbre

In [5]:
while len(ArbreSymb) > 1:
    #Fusion des noeuds de poids plus faibles
    symbfusionnes = ArbreSymb[0][0] + ArbreSymb[1][0] 
    #Création d'un nouveau noeud
    noeud = Node(symbfusionnes)
    temp = [symbfusionnes, ArbreSymb[0][1] + ArbreSymb[1][1], noeud]
    #Ajustement de l'arbre pour connecter le nouveau avec ses parents 
    ArbreSymb[0][2].parent = noeud
    ArbreSymb[1][2].parent = noeud
    #Enlève les noeuds fusionnés de la liste de noeud à fusionner.
    del ArbreSymb[0:2]
    #Ajout du nouveau noeud à la liste et tri.
    ArbreSymb += [temp]
    #Pour affichage de l'arbre ou des sous-branches
    print('\nArbre actuel:\n\n')
    for i in range(len(ArbreSymb)):
        if len(ArbreSymb[i][0]) > 1:
            print(RenderTree(ArbreSymb[i][2], style=AsciiStyle()).by_attr())   
    ArbreSymb = sorted(ArbreSymb, key=lambda x: x[1])  
    print(ArbreSymb)


Arbre actuel:


ED
|-- E
+-- D
[['F', 10, Node('/F')], ['C', 10, Node('/C')], ['ED', 12, Node('/ED')], ['A', 15, Node('/A')], ['B', 20, Node('/B')]]

Arbre actuel:


ED
|-- E
+-- D
FC
|-- F
+-- C
[['ED', 12, Node('/ED')], ['A', 15, Node('/A')], ['B', 20, Node('/B')], ['FC', 20, Node('/FC')]]

Arbre actuel:


FC
|-- F
+-- C
EDA
|-- ED
|   |-- E
|   +-- D
+-- A
[['B', 20, Node('/B')], ['FC', 20, Node('/FC')], ['EDA', 27, Node('/EDA')]]

Arbre actuel:


EDA
|-- ED
|   |-- E
|   +-- D
+-- A
BFC
|-- B
+-- FC
    |-- F
    +-- C
[['EDA', 27, Node('/EDA')], ['BFC', 40, Node('/BFC')]]

Arbre actuel:


EDABFC
|-- EDA
|   |-- ED
|   |   |-- E
|   |   +-- D
|   +-- A
+-- BFC
    |-- B
    +-- FC
        |-- F
        +-- C
[['EDABFC', 67, Node('/EDABFC')]]


On traverse l'arbre des symboles par parcours préfix, et on construit un arbre semblable de codes binaires. Le premier enfant avec 0, le deuxième avec 1.  Ensuite, on utilisera les 2 arbres pour construire le dictionnaire. 

In [6]:
ArbreCodes = Node('')
noeud = ArbreCodes
#print([node.name for node in PreOrderIter(ArbreSymb[0][2])])
parcoursprefix = [node for node in PreOrderIter(ArbreSymb[0][2])]
parcoursprefix = parcoursprefix[1:len(parcoursprefix)] #ignore la racine

Prevdepth = 0 #pour suivre les mouvements en profondeur dans l'arbre
for node in parcoursprefix:  #Liste des noeuds 
    if Prevdepth < node.depth: #On va plus profond dans l'arbre, on met un 0
        temp = Node(noeud.name + '0')
        noeud.children = [temp]
        if node.children: #On avance le "pointeur" noeud si le noeud ajouté a des enfants.
            noeud = temp
    elif Prevdepth == node.depth: #Même profondeur, autre feuille, on met un 1
        temp = Node(noeud.name + '1')
        noeud.children = [noeud.children[0], temp]  #Ajoute le deuxième enfant
        if node.children: #On avance le "pointeur" noeud si le noeud ajouté a des enfants.
            noeud = temp
    else:
        for i in range(Prevdepth-node.depth): #On prend une autre branche, donc on met un 1
            noeud = noeud.parent #On remontre dans l'arbre pour prendre la prochaine branche non explorée.
        temp = Node(noeud.name + '1')
        noeud.children = [noeud.children[0], temp]
        if node.children:
            noeud = temp
        
    Prevdepth = node.depth    
    
print('\nArbre des codes:\n\n',RenderTree(ArbreCodes, style=AsciiStyle()).by_attr())         
print('\nArbre des symboles:\n\n', RenderTree(ArbreSymb[0][2], style=AsciiStyle()).by_attr())                


Arbre des codes:

 
|-- 0
|   |-- 00
|   |   |-- 000
|   |   +-- 001
|   +-- 01
+-- 1
    |-- 10
    +-- 11
        |-- 110
        +-- 111

Arbre des symboles:

 EDABFC
|-- EDA
|   |-- ED
|   |   |-- E
|   |   +-- D
|   +-- A
+-- BFC
    |-- B
    +-- FC
        |-- F
        +-- C


In [7]:
ArbreSymbList = [node for node in PreOrderIter(ArbreSymb[0][2])]
ArbreCodeList = [node for node in PreOrderIter(ArbreCodes)]

for i in range(len(ArbreSymbList)):
    if ArbreSymbList[i].is_leaf: #Génère des codes pour les feuilles seulement
        temp = list(filter(lambda x: x[0] == ArbreSymbList[i].name, dictionnaire))
        if temp:
            indice = dictionnaire.index(temp[0])
            dictionnaire[indice][1] = ArbreCodeList[i].name
            
print(dictionnaire) 

[['E', '000'], ['B', '10'], ['F', '110'], ['A', '01'], ['C', '111'], ['D', '001']]


Codage du message avec le dictionnaire

In [8]:
MessageCode = []
longueur = 0 
for i in range(len(Message)):
    substitution = list(filter(lambda x: x[0] == Message[i], dictionnaire))
    MessageCode += [substitution[0][1]]
    longueur += len(substitution[0][1]) 

In [9]:
print(MessageCode)

['000', '10', '110', '110', '10', '000', '01', '10', '000', '110', '111', '110', '001', '000', '10', '10', '110', '110', '110', '000', '110', '000', '110', '111', '111', '10', '110', '01', '111', '10', '10', '01', '10', '111', '001', '01', '111', '001', '111', '01', '10', '01', '10', '10', '10', '111', '001', '001', '111', '001', '111', '01', '01', '01', '10', '10', '10', '01', '10', '01', '10', '01', '01', '10', '01', '01', '10']


Longueur en bits du message codé et celle de l'original

In [10]:
print("Longueur = {0}".format(longueur))
print("Longueur originale = {0}".format(longueurOriginale))

Longueur = 166
Longueur originale = 201.0
